In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np 
import pandas as pd
import os

import chart_studio.plotly as py
from plotly import tools
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=False)
import plotly.figure_factory as ff
import plotly.graph_objs as go

print(os.listdir("../input"))


In [ ]:
deliveries = pd.read_csv('../input/ipldata/deliveries.csv')
matches = pd.read_csv('../input/ipldata/matches.csv')



In [ ]:
matches.drop(['umpire3'],axis=1)

In [ ]:
matches.season.value_counts().head(10)

In [ ]:
matches_played=pd.concat([matches['team1'],matches['team2']])
matches_played=matches_played.value_counts().reset_index()
matches_played.columns=['teams','Matches played']
matches_played['wins']=matches['winner'].value_counts().reset_index()['winner']
matches_played.set_index(['teams'],inplace=True)

In [ ]:
matches_played.reset_index().head()

In [ ]:
winpercentage=round(matches_played['wins']/matches_played['Matches played'],3)*100

In [ ]:
print(type(winpercentage))


In [ ]:
winpercentage.head(3)

In [ ]:
winpercentage

In [ ]:
venue_matches=matches.groupby('venue').count()[['id']].sort_values(by='id',ascending=False)
venue_matches.head(3)
ser=pd.Series(venue_matches['id'])

In [ ]:
ser

In [ ]:
mithil1=matches[(matches['winner']=='Mumbai Indians')& (matches['venue']=='Wankhede Stadium')&((matches.team1=='Mumbai Indians')|( matches.team2=='Mumbai Indians'))].count()[['id']]
mithil=matches[(matches['winner']!='Mumbai Indians')& (matches['venue']=='Wankhede Stadium')&((matches.team1=='Mumbai Indians')|( matches.team2=='Mumbai Indians'))].count()[['id']]

In [ ]:
print(mithil1/(mithil1+mithil))

In [ ]:
ump=pd.concat([matches['umpire1'],matches['umpire2']])

umps=ump.value_counts()
umps=pd.DataFrame(umps)
umps.columns=['count']

umps.head(3)


In [ ]:
batsmen = matches[['id','season']].merge(deliveries, left_on = 'id', right_on = 'match_id', how = 'left').drop('id', axis = 1)

In [ ]:
(batsmen).head(3)

In [ ]:
season=batsmen.groupby(['season'])['total_runs'].sum().reset_index()


In [ ]:
season

In [ ]:
average_run=matches.groupby('season').count().id.reset_index()


In [ ]:
average_run

In [ ]:
average_run.rename(columns={'id':'matches'},inplace=True)

In [ ]:
average_run['total_runs']=season['total_runs']

In [ ]:
average_run['average_runs']=average_run['total_runs']/average_run['matches']

In [ ]:
round(average_run
     
     )

In [ ]:
batsmen

In [ ]:
season_six=batsmen.groupby('season')['batsman_runs'].agg(lambda x : (x==6).sum()).reset_index()

In [ ]:
season_four=batsmen.groupby('season')['batsman_runs'].agg(lambda x : (x==4).sum()).reset_index()
season_six=season_six.merge(season_four,left_on='season',right_on='season',how='left')
season_six

In [ ]:
season_six=season_six.rename(columns={'batsman_runs_x':'6','batsman_runs_y':'4'})

In [ ]:
print(season_six)

In [ ]:
high_scores=deliveries.groupby(['match_id', 'inning','batting_team','bowling_team'])['total_runs'].sum().reset_index() 
high_scores=high_scores[high_scores['total_runs']>=200]
high_scores.nlargest(10,'total_runs')

In [ ]:
new=high_scores.groupby('batting_team').count().match_id.reset_index()


In [ ]:
new1=new.sort_values(by='match_id',ascending=False).reset_index()


In [ ]:
new1.drop(['index'], axis=1)

In [ ]:
high_scores

In [ ]:
high_scores=high_scores.groupby(['inning','batting_team']).count().reset_index()
high_scores.drop(["bowling_team","total_runs"],axis=1,inplace=True)
high_scores.rename(columns={"match_id":"total_times"},inplace=True)

high_scores_1 = high_scores[high_scores['inning']==1]
high_scores_2 = high_scores[high_scores['inning']==2]


In [ ]:
high_scores_1.sort_values(by = 'total_times',ascending=False).head(2)



In [ ]:
high_scores_1

In [ ]:
print(type(high_scores_1))

In [ ]:
slices=high_scores1['is_score_chased'].value_counts().reset_index().is_score_chased

In [ ]:
agg = matches[['id','season', 'winner', 'toss_winner', 'toss_decision', 'team1']].merge(deliveries, left_on = 'id', right_on = 'match_id', how = 'left').drop('id', axis = 1)

In [ ]:
agg

In [ ]:
batsman_grp = agg.groupby(["season","match_id", "inning", "batting_team", "batsman"])

In [ ]:
batsman_grp

In [ ]:
batsmen = batsman_grp["batsman_runs"].sum().reset_index()

In [ ]:
batsmen

In [ ]:
runs_scored = batsmen.groupby(['season','batting_team', 'batsman'])['batsman_runs'].agg(['sum','mean']).reset_index()

In [ ]:
runs_per_over = deliveries.pivot_table(index=['over'],columns='batting_team',values='total_runs',aggfunc=sum)
runs_per_over.reset_index(inplace=True)
runs_per_over.drop(['Kochi Tuskers Kerala','Pune Warriors','Rising Pune Supergiant','Gujarat Lions'],axis=1,inplace=True)

In [ ]:
runs_per_over

In [ ]:
season=matches[['id','season','winner']]
complete_data=deliveries.merge(season,how='inner',left_on='match_id',right_on='id')
powerplay_data=complete_data[complete_data['over']<=6]
inn1 = powerplay_data[ powerplay_data['inning']==1].groupby('match_id')['total_runs'].agg(['sum']).reset_index()
inn2 = powerplay_data[ powerplay_data['inning']==2].groupby('match_id')['total_runs'].agg(['sum']).reset_index()


In [ ]:
inn2

In [ ]:
pi1=powerplay_data[ powerplay_data['inning']==1].groupby(['season','match_id'])['total_runs'].agg(['sum'])
pi2=powerplay_data[ powerplay_data['inning']==2].groupby(['season','match_id'])['total_runs'].agg(['sum'])
pi1=pi1.reset_index().groupby('season')['sum'].mean()
pi1=pi1.to_frame().reset_index()
pi2=pi2.reset_index().groupby('season')['sum'].mean()
pi2=pi2.to_frame().reset_index()

In [ ]:
mithil=pi2.merge(pi1,how='left',left_on='season',right_on='season')

In [ ]:
mithil

In [ ]:


powerplay_dismissals=powerplay_data.dropna(subset=['dismissal_kind']).groupby(['season','match_id','inning'])['dismissal_kind'].agg(['count']).reset_index().groupby('season')['count'].max()
powerplay_dismissals=powerplay_dismissals.reset_index()

In [ ]:
powerplay_dismissals

In [ ]:
deliveries.columns

In [ ]:
df_batsman_stat=deliveries[['ball','batsman_runs','match_count','average_score','six','four']]

In [ ]:
df_batsman_stat.rename(columns = {'ball' : 'ball', 'strike rate':'strike_rate','batsman run' : 'batsman_run',
                                  'match count' : 'match_count','average score' : 'average_score' ,'batsman_runs_x' :'six',
                                  'batsman_runs_y':'four'},inplace=True)
df_batsman_stat['strike_rate'] = df_batsman_stat['strike_rate']*100
df_batsman_stat.sort_values(by='batsman_run',ascending=False,inplace=True)
#df_batsman_stat.sort_values(by='batsman_run',ascending=False)
df_batsman_stat.reset_index(inplace=True)


In [ ]:
toppers=deliveries.groupby(['batsman','batsman_runs'])['total_runs'].count().reset_index()
toppers=toppers.pivot('batsman','batsman_runs','total_runs')
toppers.reset_index(inplace=True)


In [ ]:
toppers

In [ ]:
orange=matches[['id','season']]
orange=orange.merge(deliveries,left_on='id',right_on='match_id',how='left')
orange=orange.groupby(['season','batsman'])['batsman_runs'].sum().reset_index()
orange=orange.sort_values('batsman_runs',ascending=0)
orange=orange.drop_duplicates(subset=["season"],keep="first")

In [ ]:
orange.groupby('batsman')['batsman_runs'].count().sort_values(ascending=False)

In [ ]:
bowlers=deliveries.groupby('bowler').sum().reset_index()
bowl=deliveries['bowler'].value_counts().reset_index()
bowlers=bowlers.merge(bowl,left_on='bowler',right_on='index',how='left')
bowlers=bowlers[['bowler_x','total_runs','bowler_y']]
bowlers.rename({'bowler_x':'bowler','total_runs':'runs_given','bowler_y':'balls'},axis=1,inplace=True)
bowlers['overs']=(bowlers['balls']//6)
bowlers_top=bowlers.sort_values(by='runs_given',ascending=False)
bowlers_top=bowlers_top.head(20)


In [ ]:
bowlers_top

In [ ]:
dismissal_kinds = ["bowled", "caught", "lbw", "stumped", "caught and bowled", "hit wicket"]  #since run-out is not creditted to the bowler
purple=deliveries[deliveries["dismissal_kind"].isin(dismissal_kinds)]
purple=purple.merge(matches,left_on='match_id',right_on='id',how='outer')
purple=purple.groupby(['season','bowler'])['dismissal_kind'].count().reset_index()
purple=purple.sort_values(by='dismissal_kind',ascending=False)
purple=purple.drop_duplicates(subset=["season"],keep="first")

In [ ]:
purple

In [ ]:
season_winner=matches.drop_duplicates(subset=['season'], keep='last')[['season','winner']].reset_index(drop=True)

In [ ]:
season_winner

In [ ]:
season_winner = season_winner['winner'].value_counts()

season_winner = season_winner.to_frame()
season_winner.reset_index(inplace=True)
season_winner.rename(columns={'index':'team'},inplace=True)

In [ ]:
season_winner